In [1]:
!curl -O https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4073  100  4073    0     0  17348      0 --:--:-- --:--:-- --:--:-- 17331


In [3]:
!curl -O https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  642k  100  642k    0     0  1423k      0 --:--:-- --:--:-- --:--:-- 1425k


In [5]:
import minsearch

/Users/ishanagarwal/Agentic  AI/llm-zoomcamp/01-intro/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [6]:
import json

In [9]:
with open('documents.json','rt') as f:
    doc_raw = json.load(f)

In [13]:
documents = []

for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [15]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp'}

In [17]:
index = minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]
)

In [19]:
q = "The course has already started ,can I still enroll?"

In [21]:
index.fit(documents)

In [29]:
import os
from dotenv import load_dotenv

load_dotenv()

# Set OPENAI_API_KEY for libraries that look for it
api_key = os.getenv("API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

In [33]:
from openai import OpenAI

client = OpenAI()

In [35]:
q

'The course has already started ,can I still enroll?'

In [122]:
def search(query):
    boost = {
    'question': 3.0,
    'section': 0.5
    }

    results = index.search(
        query=query,
        filter_dict= {'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [124]:
def build_prompt(query,search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE.
    
    QUESTION : {question}
    
    CONTEXT : {context}
    
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query,context=context).strip()

    return prompt

In [126]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user','content':prompt}]
    )
    
    return response.choices[0].message.content

In [134]:
query = 'How do i run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)

    return answer

In [138]:
rag(q)

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."